# fMRIPrep + tedana + XCP-D

There are three key elements to running tedana + XCP-D:

1.  XCP-D is very strict about how it expects external (i.e., custom) confounds files to be organized and named.
    We need to copy the associated tedana derivatives (i.e., the mixing matrices) to a new folder and rename them,
    so that XCP-D can find them.
2.  Tedana's ICA will probably look bad if dummy scans haven't been dropped from the BOLD file.
    See [this discussion](https://github.com/ME-ICA/tedana/discussions/899).
    Therefore, we must drop the dummy scans before running tedana, 
    then buffer the mixing matrix produced by tedana with dummy data to fill in those scans,
    and finally run XCP-D with the same number of dummy scans flagged, 
    so those volumes will ultimately be ignored.
3.  Simply regressing out "bad" components flagged by tedana is a bad idea.
    This is known as "aggressive" denoising, and is not recommended,
    as "bad" components may share variance with "good" components that we are confident reflect real signals.
    For more information on this, 
    see [tedana's documentation](https://tedana.readthedocs.io/en/latest/denoising.html).
    You have two solid options for handling this:
    
    1.  Use the `--tedort` flag when running tedana, 
        so that "bad" components are orthogonalized with respect to "good" components.
        Unless you have manually reviewed your components and are very sure that none of the 
        "good" components reflect noise 
        (including BOLD-based noise that tedana, by its nature, cannot distinguish from real BOLD signal),
        then we recommend using this approach, and this is the one we document in this notebook.
        See [this NeuroStars post](https://neurostars.org/t/summary-report-of-xcp-d-using-multi-echo-bold-and-tedana/28304)
    2.  Allow XCP-D to orthogonalize _all_ of your nuisance regressors 
        (i.e., the tedana "bad" components and any additional confounds you choose to include in your XCP-D run) 
        with respect to the tedana "good" components.
        The potential issue with this is if your tedana results classify some noise signals as "good" components 
        (e.g., if there is a misclassification or if some components reflect BOLD-based noise, 
        which tedana cannot distinguish from BOLD-based signals).

## Step 0: Run fMRIPrep 22.0.0+ with `--me-output-echos` flag

This has already been handled for this dataset. Unfortunately, the data we use here are not publicly available yet.

## Step 1: Remove dummy scans from fMRIPrep files

In [1]:
import os

import nibabel as nib
import numpy as np
import pandas as pd
from tedana.workflows import tedana_workflow


def _flag_dummyvols(confounds_file):
    """Identify the number of dummy volumes flagged in an fMRIPrep confounds file."""
    confounds_df = pd.read_table(confounds_file)
    nss_cols = [c for c in confounds_df.columns if c.startswith("non_steady_state_outlier")]
    if nss_cols:
        initial_volumes_df = confounds_df[nss_cols]
        dummy_vols = np.any(initial_volumes_df.to_numpy(), axis=1)
        dummy_vols = np.where(dummy_vols)[0]

        # reasonably assumes all NSS volumes are contiguous
        n_dummy_vols = int(dummy_vols[-1] + 1)
        # dummy_scans = 10
    else:
        n_dummy_vols = 0
    
    return n_dummy_vols


def _remove_dummyvols(in_file, out_file, n_dummy_vols):
    """Remove dummy volumes from in_file and write out to out_file.
    
    If n_dummy_vols is 0, then just return out_file.
    """
    if n_dummy_vols:
        print(f"Dropping {n_dummy_vols} volumes from {os.path.basename(in_file)}")

        img = nib.load(in_file)
        img = img.slicer[..., n_dummy_vols:]
        img.to_filename(out_file)
    else:
        out_file = in_file

    return out_file


def drop_dummy_vols(bold_files, confounds_file, temp_dir="."):
    """Remove dummy volumes from a list of files.
    
    This infers the number of dummy volumes from the confounds file.
    The shortened files are written out to temp_dir with the same
    filenames as the original files.
    """
    shortened_files = []
    n_dummy_vols = _flag_dummyvols(confounds_file)
    for bold_file in bold_files:
        if n_dummy_vols:
            temp_file = os.path.join(temp_dir, os.path.basename(bold_file))
            shortened_file = _remove_dummyvols(bold_file, temp_file, n_dummy_vols)
        else:
            shortened_file = bold_file
        shortened_files.append(shortened_file)

    return shortened_files, n_dummy_vols


## Step 2: Run tedana

In [2]:
dset_dir = "/Users/taylor/Documents/datasets/xcpd_example"
deriv_dir = os.path.join(dset_dir, "derivatives")
func_dir = os.path.join(deriv_dir, "fmriprep/sub-06/ses-2/func")

ECHO_TIMES = [14.2, 38.93, 63.66, 88.39, 113.12]  # hardcoded bc i'm lazy
prefix = f"sub-06_ses-2_task-frackack_acq-multiecho"
bold_files = [
    os.path.join(func_dir,f"{prefix}_echo-{echo + 1}_part-mag_desc-preproc_bold.nii.gz")
    for echo in range(len(ECHO_TIMES))
]
confounds_file = os.path.join(func_dir,  f"{prefix}_part-mag_desc-confounds_timeseries.tsv")
mask_file = os.path.join(func_dir, f"{prefix}_part-mag_desc-brain_mask.nii.gz")

# Write tedana outputs to BIDS-like structure,
# but use a separate folder for each run.
tedana_out_dir = os.path.join(deriv_dir, "tedana/sub-06/ses-2/func", prefix)
os.makedirs(tedana_out_dir, exist_ok=True)
# A folder for all of the shortened files.
tedana_temp_dir = os.path.join(dset_dir, "derivatives", "reduced_files")
os.makedirs(tedana_temp_dir, exist_ok=True)

shortened_files, n_dummy_vols = drop_dummy_vols(
    bold_files=bold_files,
    confounds_file=confounds_file,
    temp_dir=tedana_temp_dir,
)

tedana_workflow(
    data=shortened_files,
    tes=ECHO_TIMES,
    out_dir=tedana_out_dir,
    mask=mask_file,
    prefix=prefix,
    fittype="loglin",
    tedort=True,
)

Dropping 2 volumes from sub-06_ses-2_task-frackack_acq-multiecho_echo-1_part-mag_desc-preproc_bold.nii.gz
Dropping 2 volumes from sub-06_ses-2_task-frackack_acq-multiecho_echo-2_part-mag_desc-preproc_bold.nii.gz
Dropping 2 volumes from sub-06_ses-2_task-frackack_acq-multiecho_echo-3_part-mag_desc-preproc_bold.nii.gz
Dropping 2 volumes from sub-06_ses-2_task-frackack_acq-multiecho_echo-4_part-mag_desc-preproc_bold.nii.gz
Dropping 2 volumes from sub-06_ses-2_task-frackack_acq-multiecho_echo-5_part-mag_desc-preproc_bold.nii.gz


INFO     tedana:tedana_workflow:483 Using output directory: /Users/taylor/Documents/datasets/xcpd_example/derivatives/tedana/sub-06/ses-2/func/sub-06_ses-2_task-frackack_acq-multiecho
INFO     tedana:tedana_workflow:501 Loading input data: ['/Users/taylor/Documents/datasets/xcpd_example/derivatives/reduced_files/sub-06_ses-2_task-frackack_acq-multiecho_echo-1_part-mag_desc-preproc_bold.nii.gz', '/Users/taylor/Documents/datasets/xcpd_example/derivatives/reduced_files/sub-06_ses-2_task-frackack_acq-multiecho_echo-2_part-mag_desc-preproc_bold.nii.gz', '/Users/taylor/Documents/datasets/xcpd_example/derivatives/reduced_files/sub-06_ses-2_task-frackack_acq-multiecho_echo-3_part-mag_desc-preproc_bold.nii.gz', '/Users/taylor/Documents/datasets/xcpd_example/derivatives/reduced_files/sub-06_ses-2_task-frackack_acq-multiecho_echo-4_part-mag_desc-preproc_bold.nii.gz', '/Users/taylor/Documents/datasets/xcpd_example/derivatives/reduced_files/sub-06_ses-2_task-frackack_acq-multiecho_echo-5_part-mag_d

## Step 3: Label tedana components and fill in dummy volumes

In [3]:
# Load (orthogonalized) mixing matrix and classifications from tedana
mixing_matrix = os.path.join(tedana_out_dir, f"{prefix}_desc-ICAOrth_mixing.tsv")
metrics_df = os.path.join(tedana_out_dir, f"{prefix}_desc-tedana_metrics.tsv")
mixing_matrix = pd.read_table(mixing_matrix)
metrics_df = pd.read_table(metrics_df)

# Prepend "signal__" to all accepted components' column names
rejected_columns = metrics_df.loc[metrics_df["classification"] == "rejected", "Component"]
mixing_matrix = mixing_matrix[rejected_columns]

# Add dummyvols back in to beginning of matrix, as copies of the first row
mixing_matrix_data = mixing_matrix.to_numpy()
first_row = mixing_matrix_data[0, :]
leading_rows = np.ones((n_dummy_vols, mixing_matrix.shape[1])) * first_row
new_mixing_matrix_arr = np.vstack((leading_rows, mixing_matrix_data))
new_mixing_matrix = pd.DataFrame(new_mixing_matrix_arr, columns=mixing_matrix.columns)

# Write out to custom confounds folder
custom_confounds_folder = os.path.join(deriv_dir, "custom_confounds_for_xcpd")
os.makedirs(custom_confounds_folder, exist_ok=True)
# use the same name as the fMRIPrep confounds, but in the new folder
custom_confounds_file = os.path.join(custom_confounds_folder, os.path.basename(confounds_file))
new_mixing_matrix.to_csv(custom_confounds_file, sep="\t", index=False)

## Step 4: Run XCP-D with tedana-derived custom confounds

Note: dummy-scans must match between tedana and XCP-D
```bash
docker run --rm -u $(id -u) \
    -v /Users/taylor/Documents/datasets/ds003643:/bids-input:rw \
    -v /Users/taylor/Documents/tsalo/xcp_d/xcp_d:/usr/local/miniconda/lib/python3.8/site-packages/xcp_d \
    -v /Users/taylor/Documents/tsalo/xcp_d_testing/data/license.txt:/license.txt --env FS_LICENSE=/license.txt \
    pennlinc/xcp_d:unstable \
    /bids-input/derivatives/fmriprep \
    /bids-input/derivatives \
    participant \
    -w /bids-input/derivatives/work \
    --participant_label EN100 \
    --nuisance-regressors 27P \
    --custom_confounds /bids-input/derivatives/custom_confounds_for_xcpd \
    --dummy-scans auto \
    --bids-filter-file /bids-input/derivatives/code/filter_file.json \
    -vvv
```